<a href="https://colab.research.google.com/github/Himanshu0518/NLP/blob/main/Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 95% 77.0M/80.9M [00:01<00:00, 80.0MB/s]
100% 80.9M/80.9M [00:01<00:00, 70.0MB/s]


In [ ]:
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'
with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='latin-1')

In [ ]:
df.columns = ['sentiments','id','date','query','user','text']

In [ ]:
df.head()

,sentiments,id,date,query,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
df.tail()

,sentiments,id,date,query,user,text
1599994,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599995,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599996,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599997,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...
1599998,4,2193602129,Tue Jun 16 08:40:50 PDT 2009,NO_QUERY,RyanTrevMorris,happy #charitytuesday @theNSPCC @SparksCharity...


In [ ]:
df.shape

(1599999, 6)



*   0 -- > For negative reviews
*   4 --> For positive reviews



Checking for Missing values

In [ ]:
df.isnull().sum()

,0
sentiments,0
id,0
date,0
query,0
user,0
text,0


In [ ]:
df['query'].value_counts()

,count
query,
NO_QUERY,1599999


In [ ]:
df = df.drop(['id','date','query','user'] , axis = 1)

In [ ]:
sum(df['text'].apply(len)<5)

0

In [ ]:
df = df[df['text'].apply(len)>5]
df.shape

(1599999, 2)

In [ ]:
df.sample(10)

,sentiments,text
1263674,4,@Caaitlynn i like the pic
1592494,4,@sawyersgem I've got English on Thursday and t...
997250,4,Moving profile to --------------&gt;&gt;&gt;&g...
978177,4,No updates for 3 days... Kind of like fasting ...
824862,4,@Michy_Boo So far so good
1329263,4,"@sheeradjafar hmm die x ckp, but shud be soon."
353074,0,@SophBrassington Are they nice? Awww i'm getti...
113087,0,i totally think going to bed late was a bad op...
389773,0,Cows hate walking on cement i did it for the ...
1576987,4,@sassisam Thats okay Good luck finding someone!


In [ ]:
df.head()

,sentiments,text
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@Kwesidei not the whole crew


In [ ]:
df.sentiments.value_counts()

,count
sentiments,
4,800000
0,799999


Replace the positive reviews by 1

In [ ]:
df['sentiments'] = df['sentiments'].replace({4:1})

In [ ]:
import nltk
from nltk.corpus import stopwords
import re
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stopwords.words('english')[0:5]

['i', 'me', 'my', 'myself', 'we']

In [ ]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

In [ ]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [stemmer.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [ ]:
stemming('@Kenichan I dived many times for the ball. Man...')

'kenichan dive mani time ball man'

In [ ]:
df['text'] = df['text'].apply(stemming)

In [ ]:
df.head()

,sentiments,text
0,0,upset updat facebook text might cri result sch...
1,0,kenichan dive mani time ball manag save rest g...
2,0,whole bodi feel itchi like fire
3,0,nationwideclass behav mad see
4,0,kwesidei whole crew


In [ ]:
X = df['text'].values
Y = df['sentiments'].values

splitting the data into train and text data

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.001,random_state = 2)

In [ ]:
X_train.shape , X_test.shape

((1598399,), (1600,))

# TF-IDF Vectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

In [ ]:
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)  # Apply on raw text data
X_test_tfidf = vectorizer.transform(X_test)        # Apply on raw text data


In [ ]:
vocab = vectorizer.get_feature_names_out()
print("Vocabalory: ",vocab)

Vocabalory:  ['aa' 'aaaaaag' 'aaaaw' ... 'zm' 'zuckerbabi' 'zuzufalta']


In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
classifier = LogisticRegression()
classifier.fit(X_train_tfidf,Y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
y_pred = classifier.predict(X_test_tfidf)
accuracy_score(Y_test,y_pred)

0.7625

# LSTM and tensorflow


In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense,Dropout

In [ ]:
vocab_size = 10000

# One Hot Representation

In [ ]:
message = X.copy()

In [ ]:
message.size

1599999

In [ ]:
onehot_rep = [one_hot(words,vocab_size) for words in message]
onehot_rep[0:5]

[[2874, 6025, 2417, 5335, 4008, 173, 1103, 9316, 5730, 9532, 7914],
 [1083, 5082, 7923, 4739, 5504, 1110, 427, 1511, 7049, 7350],
 [1326, 9369, 2462, 9940, 4296, 2677],
 [4351, 2111, 334, 3590],
 [3763, 1326, 182]]

In [ ]:
print(message[0])
onehot_rep[0]

upset updat facebook text might cri result school today also blah


[2874, 6025, 2417, 5335, 4008, 173, 1103, 9316, 5730, 9532, 7914]

# Embedding representation

In [ ]:
sent_length = 20
embedded_docs = pad_sequences(onehot_rep,padding='pre',maxlen=sent_length)

In [ ]:
print(embedded_docs[0])
print(type(embedded_docs[0]))

[   0    0    0    0    0    0    0    0    0 2874 6025 2417 5335 4008
  173 1103 9316 5730 9532 7914]
<class 'numpy.ndarray'>


In [ ]:
# Creating the model
embedding_vector_features = 50
model = Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(150))
model.add(Dropout(0.3))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss='binary_crossentropy' , optimizer = 'adam' , metrics =['accuracy'])

# Build the model by specifying the input shape
model.build((None, sent_length))

model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 20, 50)              │         500,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 20, 50)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 150)                 │         120,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 150)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             151 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 620,751 (2.37 MB)

 Trainable params: 620,751 (2.37 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
X_final = np.array(embedded_docs)
Y_final = np.array(Y)

In [ ]:
Y_final

array([0, 0, 0, ..., 1, 1, 1])

In [ ]:
X_final.shape , Y_final.shape

((1599999, 20), (1599999,))

In [ ]:
train_data,test_data,train_label,test_label = train_test_split(X_final,Y_final,test_size=0.2,random_state = 4)

In [ ]:
model.fit(train_data,train_label,validation_data=(test_data,test_label),epochs=10,batch_size=64)

Epoch 1/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 1072s 53ms/step - accuracy: 0.7451 - loss: 0.5113 - val_accuracy: 0.7663 - val_loss: 0.4797
Epoch 2/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 1107s 54ms/step - accuracy: 0.7725 - loss: 0.4725 - val_accuracy: 0.7693 - val_loss: 0.4759
Epoch 3/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 1123s 55ms/step - accuracy: 0.7792 - loss: 0.4619 - val_accuracy: 0.7722 - val_loss: 0.4712
Epoch 4/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 1136s 56ms/step - accuracy: 0.7851 - loss: 0.4519 - val_accuracy: 0.7739 - val_loss: 0.4690
Epoch 5/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 1078s 54ms/step - accuracy: 0.7893 - loss: 0.4463 - val_accuracy: 0.7746 - val_loss: 0.4696
Epoch 6/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 1155s 57ms/step - accuracy: 0.7934 - loss: 0.4399 - val_accuracy: 0.7741 - val_loss: 0.4714
Epoch 7/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 1155s 56ms/step - accuracy: 0.7950 - loss: 0.4357 - val_accuracy: 0.7747 - val_loss: 0.4703
Epoch 8/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 1129

In [ ]:
print(Y_final)

[0 0 0 ... 1 1 1]


In [ ]:
y_pred = model.predict(test_data)

10000/10000 ━━━━━━━━━━━━━━━━━━━━ 130s 13ms/step


In [ ]:
y_pred = np.where(y_pred > 0.5,1,0)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
confusion_matrix(test_label,y_pred)

array([[123542,  36517],
       [ 35998, 123943]])

In [ ]:
accuracy_score(y_pred,test_label)

0.773390625

In [ ]:
import pickle

In [ ]:
filename = 'sentiment_model.sav'
pickle.dump(model,open(filename,'wb'))

In [ ]:
loaded_model = pickle.load(open('sentiment_model.sav','rb'))

In [ ]:
Y_pred = loaded_model.predict(test_data)